# Analizando la página objetivo

Este notebook tiene como objetivo analizar la página a la cual queremos hacer webscrapping:

https://www.isciii.es/QueHacemos/Servicios/VigilanciaSaludPublicaRENAVE/EnfermedadesTransmisibles/Paginas/InformesCOVID-19.aspx

Esta página contiene los datos actualizados por parte del gobierno de los casos de contagio por coronavirus. El obejtivo de este notebook es en primer lugar, realziar webscrapping para descargar los PDFs que redirige. Seguidamente, para recolectar los datos del pdf realizaremos pdf scraping. Todos los pasos se automatizan también en scripts en el proyecto.

In [1]:
# libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup
from bs4.element import Tag

In [2]:
# Descargamos la página objetivo
page = requests.get(
    'https://www.isciii.es/QueHacemos/Servicios/VigilanciaSaludPublicaRENAVE/'
    'EnfermedadesTransmisibles/Paginas/InformesCOVID-19.aspx'
)
if page.status_code == 200:
    print('Página descargada con éxito')

Página descargada con éxito


In [3]:
# creamos la sopa a traves de la página
soup = BeautifulSoup(page.content)

Las páginas objetivo se encuentran dentro de un a href.

In [4]:
# # buscamos bajo el id donde están los links de interés
# id_links = soup.find(id='ctl00_PlaceHolderMain_ctl02__ControlWrapper_RichHtmlField')

# # para cada hijo buscamos a y guardamos su enlace
# links_pdfs = []
# for child in id_links.children:
#     a_tag = child.find('a')
#     if isinstance(a_tag, Tag):
#         links_pdfs.append(a_tag.get('href'))

# print(links_pdfs[0])

In [5]:
# buscamos bajo el id donde están los links de interés
id_links = soup.find(id='ctl00_PlaceHolderMain_ctl02__ControlWrapper_RichHtmlField')

# para cada hijo buscamos a y guardamos su enlace
pdfs = []
for child in id_links.children:
    pdf = {}
    a_tag = child.find('a')
    if isinstance(a_tag, Tag):
        pdf['link'] = a_tag.get('href')
        pdf['info'] = a_tag.text
        pdfs.append(pdf)

from pprint import pprint
pprint(pdfs[0])

{'info': '- Informe nº 21. Situación de COVID-19 en España a\xa06 de abril de '
         '2020',
 'link': '/QueHacemos/Servicios/VigilanciaSaludPublicaRENAVE/EnfermedadesTransmisibles/Documents/INFORMES/Informes%20COVID-19/Informe%20nº%2021.%20Situación%20de%20COVID-19%20en%20España%20a%206%20de%20abril%20de%202020.pdf'}


In [6]:
# generamos cada enlace
baselink = 'https://www.isciii.es'
for pdf in pdfs:
    link = pdf['link']
    pdf.update({'link': baselink + link})
pprint(pdfs[0])

{'info': '- Informe nº 21. Situación de COVID-19 en España a\xa06 de abril de '
         '2020',
 'link': 'https://www.isciii.es/QueHacemos/Servicios/VigilanciaSaludPublicaRENAVE/EnfermedadesTransmisibles/Documents/INFORMES/Informes%20COVID-19/Informe%20nº%2021.%20Situación%20de%20COVID-19%20en%20España%20a%206%20de%20abril%20de%202020.pdf'}


In [7]:
## generamos cada enlace
#baselink = 'https://www.isciii.es'
#for i in range(0, len(links_pdfs)):
    #links_pdfs[i] = baselink + links_pdfs[i]
#print(links_pdfs[0])

In [8]:
name = pdfs[0]['info']
url = pdfs[0]['link']
file = requests.get(url, allow_redirects=True)
open('../data/' + name + '.pdf', 'wb').write(file.content)

1490624

Ahora que ya hemos descargado un pdf vamos a hacer pdf scrapping para extraer los datos.

In [9]:
import tabula
contenido_pdf = tabula.read_pdf('../data/' + name + '.pdf', pages='all', output_format='json')

Error from tabula-java:
Error: could not open `D:\Java JDK\lib\amd64\jvm.cfg'




CalledProcessError: Command '['java', '-Dfile.encoding=UTF8', '-jar', "d:\\googledrive\\master's degree data science\\tipología y ciclo de vida de los datos\\práctica 1 - webscrapping\\webscrap_pract_1\\venv\\lib\\site-packages\\tabula\\tabula-1.0.3-jar-with-dependencies.jar", '--pages', 'all', '--guess', '--format', 'JSON', '../data/- Informe nº 21. Situación de COVID-19 en España a\xa06 de abril de 2020.pdf']' returned non-zero exit status 1.

In [ ]:
# let's look for the interesting table
comunidades = [
    'Andalucía',
    'Aragón',
    'Asturias',
    'Baleares',
    'Canarias',
    'Cantabria',
    'Castilla La Mancha',
    'Castilla y León',
    'Cataluña',
    'Comunitat Valenciana',
    'Extremadura',
    'Galicia',
    'Madrid',
    'Murcia',
    'Navarra',
    'País Vasco',
    'La Rioja',
    'Ceuta',
    'Melilla'
]
datos_interes = []
for top_level_element in contenido_pdf:
    for level_1_element in top_level_element['data']:
        comunidad = {}
        for i in range(0, len(level_1_element)):
            if level_1_element[i]['text'] in comunidades:
                comunidad['comunidad'] = level_1_element[i]['text']
                comunidad['casos'] = level_1_element[i + 1]['text']
                comunidad['casos_notificados'] = level_1_element[i + 2]['text']
                datos_interes.append(comunidad)
                
datos_interes